# SF Data Project
## By Kavish Harjai
## Data Preparation

### Synopsis

The San Francisco Rent Ordinance created the San Francisco Rent Board in 1979. The Board receives petitions from the city's landlords and tenants. In some cases, the Tent Board will oversee a mediation/arbitration session between the complainant and the alleged wrongdoer.  The nature of the petition, of course, varies depending on whom the filing party is, but the Rent Board can only adjudicate issues that are covered in the rent ordinance. 

I was specifically interested in investigating tenant petitions. In a petition, a tenant can allege one or more complaint, including wrongful eviction, unfair rent increase, a decrease in housing services (necessary repairs and replacements), etc. 

The city of San Francisco keeps a CSV of the petitions sent to the Board going back to January 1997. 

https://data.sfgov.org/Housing-and-Buildings/Petitions-to-the-Rent-Board/6swy-cmkq

The data is "wide," meaning each possible complaint is its own column, and each column contains boolean values: 'True' if the particular complaint is one of the reasons for the petition, and 'False' if not. In addition to those complaint columns, there are several columns important to this analysis, including: 

* petition_id: unique identifier for each petition
* date_filed: when the petition was filed
* filing_party: did a tenant or a landlord file the petition?
* neighborhoods: in which neighborhood does the petitioner live? 

One thing that's critical to understand is that each row refers to one petition. But a single petition can include several complaints (according to the SF Rent Board website: "The tenant may combine more than one type of claim in a Tenant Petition").

I will analyze petitions submitted by tenants from 2010 to present. I ask the following questions: 

* In which year did tenants file the most petitions?
* What kinds of complaints are the most popular? 
* From which neighborhoods did most complaints arise? And how does the number of complaints compare to the population of renters in each neighborhood? 

This project is divided into two notebooks. One focused on data preparation and another focused on analysis. Both can be found in the notebooks subfolder. My findings are located in the analysis notebook.

In this data preparation notebook, I prepare the data in three different chunks. In **Misc,** I import the csv and do a top-level analysis to understand the kind of data I'm working with. In **Filtering,** I whittle the dataframe down to the rows that will help me answer the above questions. In **Melt,** I reshape the data to be more conducive to analysis. 

## Misc

#### Import necessary libraries.

In [37]:
import pandas as pd
from datetime import datetime as dt
import os
import numpy as np

In [38]:
#pd.set_option('display.max_columns', None)
#Optional command to view all columns! 

#### Set up relative paths so any user can run the analysis on their own machine.

In [39]:
data_dir = os.environ["DATA_DIR"]
raw_data = data_dir + "/raw/"
processed_data = data_dir + '/processed/'
petitions_file = os.path.join(raw_data, 'Petitions_to_the_Rent_Board.csv')

#### Bring in CSV as a dataframe called 'all_petitions' and investigate the data for null values, data types, etc.

In [40]:
all_petitions = pd.read_csv(petitions_file, low_memory=False)

In [41]:
all_petitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51881 entries, 0 to 51880
Data columns (total 43 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   Petition ID                                                         51881 non-null  object 
 1   Date Filed                                                          51881 non-null  object 
 2   Priority                                                            51881 non-null  bool   
 3   Appeal ID                                                           81 non-null     object 
 4   Filing Party                                                        51869 non-null  object 
 5   Prop I Petition                                                     215 non-null    object 
 6   Ground (landlord): Capital Improvement                              51881 non-null  bool   
 7   Ground (landl

#### Overall, the data looks pretty good 😎

##### The good:

* All the tenant complaint columns are filled—that means there's either a True or False value in each of the rows.
* The petition_id and date_filed columns are also all filled out.

##### The bad:

* There are 12 missing values in the filing_party column. That's a negligible amount considering the size of the dataset, but I'll need to filter those out before I filter for just tenant complaints. 
* There are just more than 1,300 missing values for the neighborhoods column. That'll be important to consider in the next coming steps since that column is central to my questions.
* The column headers aren't cute: they're not standardized and include spaces. 

##### The ugly:

* As mentioned before, the data is wide-shaped. That will require I melt the data later. 
* Many columns (those referring to landlord complaints) are irrelevant to this analysis. 

#### One thing I need to do before I go crazy is condense and reformat the names into snake_case.

In [42]:
all_petitions = all_petitions.rename(columns={'Petition ID': 'petition_id', 
                                              'Date Filed': 'date_filed', 
                                              'Appeal ID': 'appeal_id', 
                                              'Filing Party': 'filing_party',
                                              'Prop I Petition': 'prop_i_petition',
                                             'Ground (tenant): Unlawful Rent Increase': 'tenant_rent_increase',
                                             'Petition Source Zipcode': 'petition_zip_code',
                                             'Supervisor District': 'supervisor_district',
                                             'Neighborhoods - Analysis Boundaries': 'neighborhoods',
                                           'Ground (tenant): Decrease in Housing Services': 'tenant_decrease_housing_services',
                                          'Ground (tenant): Failure to Repair and Maintain':'tenant_failure_repair_maintain',
                                          'Ground (tenant): Passthrough Challenge':'tenant_passthrough_challenge',
                                          'Ground (tenant): Summary Petition': 'tenant_summary_petition',
                                          'rent_increase_complaint':'tenant_rent_increase',
                                          'Ground (tenant): Wrongful Eviction Report':'tenant_wrongful_eviction',
                                          'Ground (tenant): Section 8 tenancy':'tenant_section_eight',
                                          'Ground (tenant): SRO Hotel Vistor Policy Petition':'tenant_hotel_visitor',
                                          'Ground (tenant): Rules and Regulations Section 6.15': 'tenant_section_six',
                                          'Ground (tenant): Other':'tenant_other',
                                          'Ground (tenant): Utility Passhtrough Hardship':'tenant_utility_passthrough',
                                          'Ground (tenant): Water Revenue Bond Passthrough Hardship':'tenant_water_revenue_hardship',
                                          'Ground (tenant): Capital Improvement Passthrough Hardship Petition':'tenant_capital_improvement',
                                          'Ground (tenant): Wrongful Severance of Housing Service':'tenant_wrongful_severance',
                                          'Ground: Alternative Dispute Resolution':'tenant_alt_dispute'
                                         }
                                )

####  I'm going to convert the date_filed column to a datetime object, so I can create two extra columns indicating in which month and year each petition was filed.

In [43]:
all_petitions['date_filed'] =  pd.to_datetime(all_petitions['date_filed'], format='%m/%d/%Y')

In [44]:
all_petitions['year'] = pd.DatetimeIndex(all_petitions['date_filed']).year
all_petitions['month'] = pd.DatetimeIndex(all_petitions['date_filed']).month

## Filtering

#### My analysis is contingent on filtering for complaints from tenants. I saw from my .info() command above that there are 12 observations where the filing party value is null. I'll remove those 12 observations. 

In [45]:
all_petitions = all_petitions[all_petitions['filing_party'].notna()]

#### Ok, I'll first filter for petitions since 2010. The date_mask object is a series of boolean values that indicates whether each row satisfies the conditions I've applied. Then I use .loc to create a new dataframe where all the boolean values=True. 

In [46]:
date_mask = (all_petitions['date_filed'] >= '2010-01-01') & (all_petitions['date_filed'] <= '2022-02-20')
petitions_since_2010 = all_petitions.loc[date_mask]

#### Checking to make sure my method work:

In [47]:
petitions_since_2010.date_filed.min()

Timestamp('2010-01-04 00:00:00')

#### Ok, nice. That worked. Let me apply the same logic to filter for petitions filed by tenants.

In [48]:
filing_party_mask = petitions_since_2010['filing_party'] == 'tenant'
tenant_petitions_since_2010 = petitions_since_2010.loc[filing_party_mask]

#### Checking myself again. Gonna do some math and see if my filtering matches up:

* X = petitions since 2010
* Y = petitions filed by landlords since 2010

#### If X - Y = the amount of observations in the tenants-only dataframe, then I know I filtered correctly.

In [49]:
X = len(petitions_since_2010)

In [50]:
Y = len(petitions_since_2010[petitions_since_2010['filing_party'] == 'landlord'])

In [51]:
len(tenant_petitions_since_2010)

16171

In [52]:
X - Y 

16171

#### Woot! Let's keep going. 

#### Ok, here I remember that there are some missing values for neighborhoods to deal with. Now that I've filtered the dataframe for only tenant petitions since 2010, let's see how many neighborhood values are missing. Recall that in the OG dataframe, more than 1,300 were missing. 

In [53]:
tenant_petitions_since_2010.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16171 entries, 841 to 26099
Data columns (total 45 columns):
 #   Column                                                         Non-Null Count  Dtype         
---  ------                                                         --------------  -----         
 0   petition_id                                                    16171 non-null  object        
 1   date_filed                                                     16171 non-null  datetime64[ns]
 2   Priority                                                       16171 non-null  bool          
 3   appeal_id                                                      0 non-null      object        
 4   filing_party                                                   16171 non-null  object        
 5   prop_i_petition                                                1 non-null      object        
 6   Ground (landlord): Capital Improvement                         16171 non-null  bool         

#### Ok, there are about 300 missing neighborhood values in my filtered dataframe. 

#### I still think it's a small amount compared to the size of the dataframe, so I'll eliminate the rows missing that value from the dataframe. 

In [67]:
tenant_petitions_since_2010 = tenant_petitions_since_2010[
    tenant_petitions_since_2010[
        'neighborhoods']
    .notna()]

In [68]:
tenant_petitions_since_2010.head()

,petition_id,date_filed,Priority,appeal_id,filing_party,prop_i_petition,tenant_decrease_housing_services,tenant_failure_repair_maintain,tenant_passthrough_challenge,tenant_summary_petition,...,tenant_capital_improvement,tenant_wrongful_severance,tenant_alt_dispute,Address,petition_zip_code,supervisor_district,neighborhoods,shape,year,month
1331,T211020,2021-12-02,False,NaN,tenant,NaN,False,False,False,False,...,True,False,False,1400 Block Of 09th Avenue,94122,5.0,Inner Sunset,POINT (-122.46612 37.761242),2021,12
1332,T210954,2021-11-15,False,NaN,tenant,NaN,True,False,False,False,...,False,False,False,0 Block Of 06th Street,94103,6.0,South of Market,POINT (-122.40897 37.781185),2021,11
1333,T210953,2021-11-15,False,NaN,tenant,NaN,False,False,False,False,...,False,False,False,1200 Block Of 06th Avenue,94122,5.0,Inner Sunset,POINT (-122.4632 37.765587),2021,11
1334,T210866,2021-10-13,False,NaN,tenant,NaN,True,False,False,False,...,False,False,False,100 Block Of 06th Street,94103,6.0,South of Market,POINT (-122.40843 37.780754),2021,10
1335,T210700,2021-08-26,False,NaN,tenant,NaN,False,False,False,False,...,False,False,False,0 Block Of 06th Street,94103,6.0,South of Market,POINT (-122.40897 37.781185),2021,8


#### Last step of this data preparation section is to drop the columns about landlord complaints, since they aren't relevant to my analysis. 

In [56]:
tenant_petitions_since_2010 = tenant_petitions_since_2010.drop(labels = ['Ground (landlord): Capital Improvement',
                                                                     'Ground (landlord): Comparable Rents',
                                                                     'Ground (landlord): Extension of Time for Capital Improvements',
                                                                     'Ground (landlord): Intent to Withdraw (Ellis)',
                                                                     'Ground (landlord): Other',
                                                                     'Ground (landlord): Operating & Maintenance',
                                                                     'Ground (landlord): Substantial Rehabilitation',
                                                                     'Ground (landlord):  Costa-Hawkins',
                                                                     'Ground (landlord):  Exemption',
                                                                     'Ground (landlord): Rules and Regulations Section 1.21',
                                                                     'Ground (landlord): Rules and Regulations Section 6.14',
                                                                     'Ground (landlord): Rules and Regulations Section 6.15',
                                                                     'Ground (landlord): SRO Hotel Vistor Policy Petition',
                                                                     'Ground (landlord): Utility Passthrough Petition',
                                                                     'Ground (landlord): Owner Move-In Disability Determination',
                                                                     'Ground (landlord): Utility Passthrough Worksheet',
                                                                     'Ground (landlord): Non-Comparable Rents'],
                                                           axis=1)

## Melting dataframe

#### I'll start by using list comprehension to assess which columns I'll need to melt. These are all the tenant complaint columns. Recall that the values in each of these columns is either True or False, depending if a petition is based on that complaint. 

In [70]:
[column for column in tenant_petitions_since_2010.columns if 'tenant' in column]

['tenant_decrease_housing_services',
 'tenant_failure_repair_maintain',
 'tenant_passthrough_challenge',
 'tenant_summary_petition',
 'tenant_rent_increase',
 'tenant_wrongful_eviction',
 'tenant_section_eight',
 'tenant_hotel_visitor',
 'tenant_section_six',
 'tenant_other',
 'tenant_utility_passthrough',
 'tenant_water_revenue_hardship',
 'tenant_capital_improvement',
 'tenant_wrongful_severance',
 'tenant_alt_dispute']

#### The melt function calls for an argument 'id_vars.' These are the columns that won't be pivoted to the row format. I'll use the inverse statement from my previous cell to set the 'id_vars.'

In [58]:
id_vars = [column for column in tenant_petitions_since_2010.columns if 'tenant' not in column]

#### The next cell carries the melt function out. It will take the tenant complaint columns and convert them into rows, where each column name will be inserted as a value under a different column called 'reason,' and where each True/False value will be inserted into a second column called 'value.'

In [59]:
melted_tenant_petitions = pd.melt(tenant_petitions_since_2010,
                                  id_vars=id_vars,
                                  var_name='reason',
                                  value_name='value',
                                 )

#### So basically, the melt function added hella rows because it does not discriminate between True and False values. It went from each petition being referenced in only one row to each petition being referenced in 15 rows (since there are 15 columns that we pivoted). 

#### It's not necessary to retain the rows where value = False, since that means that that complaint wasn't included in the petition. So the next cell filters the melted dataframe to those where the value in the value column = True.

In [60]:
true_values = melted_tenant_petitions[melted_tenant_petitions.value == True]

#### That means each row will now refer to a specific complaint, unlike before where each row referred to a discrete petition. This is important for two reasons. Firstly, we can expect to see more rows in the true_values dataframe, since, as discussed above, a petition can include several complaints in one. (I've also reached out to the city to clarify/confirm.)

In [61]:
len(true_values) #gonna assume for now that there are a few petitions that allege several wrongdoings...have reached out for more information

17270

#### Secondly, it's important to understand that each row refers to a single complaint because that will judge how we make conclusions about the data. Going forward, we can't discuss the data as the number of people who filed a petition. Instead, we must discuss it in terms of complaints. X is how many times complaint Y was alleged, for example.

#### Ok last step before we get to the fun stuff. Let's see how many rows are duplicated:

In [62]:
duplicates = true_values.duplicated(subset=None, keep='first').reset_index()

In [63]:
true_values[true_values.duplicated(keep=False)]

,petition_id,date_filed,Priority,appeal_id,filing_party,prop_i_petition,Address,petition_zip_code,supervisor_district,neighborhoods,shape,year,month,reason,value
144598,T191001,2019-06-21,False,NaN,tenant,NaN,200 Block Of Valencia Street,94103,8.0,Mission,POINT (-122.42233 37.769573),2019,6,tenant_other,True
145351,T191001,2019-06-21,False,NaN,tenant,NaN,200 Block Of Valencia Street,94103,8.0,Mission,POINT (-122.42233 37.769573),2019,6,tenant_other,True


#### Looks like there's only one row that's repeated in the whole dataframe. That's a relief. I'm gonna remove that one row from the dataframe. 

In [64]:
true_values= true_values.drop_duplicates(subset=None, 
                            keep='first') 

#### To quell my anxiety, I will check that my .drop_duplicates() function worked. The length of the dataframe now should equal 17270 minus one. 

In [65]:
len(true_values)

17269

#### Alright, we're good so far. I'm going to save the filtered/cleaned dataframe using the relative path I defined earlier. This will make it easy for any user to pick up at the analysis stage. 

In [66]:
true_values_path = os.path.join(processed_data, 'tenant_petitions_2010.csv')
true_values.to_csv(true_values_path, index=False)